# LazyGraphRAG Search Example

LazyGraphRAG is a cost-efficient alternative to full GraphRAG that achieves comparable quality at approximately **1/100th of the cost**. It uses iterative deepening search with budget-controlled LLM calls.

## Key Features

- **Budget-controlled LLM calls**: Z100/Z500/Z1500 presets for cost management
- **Iterative deepening search**: Progressive exploration focused on relevant content
- **Query expansion**: Subquery decomposition for comprehensive coverage
- **Claim extraction**: Structured fact extraction from relevant content

## Setup

First, let's import the necessary modules and load the indexed data.

In [ ]:
import os

import pandas as pd

from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.config.models.lazy_search_config import LazySearchConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.indexer_adapters import read_indexer_text_units
from graphrag.query.structured_search.lazy_search import (
    LazySearch,
    LazySearchData,
)

## Load Indexed Data

LazyGraphRAG primarily uses text chunks from the indexing pipeline. Unlike Local or Global search, it doesn't require the full knowledge graph - just the text units.

In [ ]:
INPUT_DIR = "./inputs/operation dulce"
TEXT_UNIT_TABLE = "text_units"

# Read text units
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")

print(f"Text unit count: {len(text_unit_df)}")
text_unit_df.head()

## Initialize Chat Model

LazyGraphRAG uses an LLM for query expansion, relevance testing, and response generation.

In [ ]:
api_key = os.environ.get("GRAPHRAG_API_KEY", "")

chat_config = LanguageModelConfig(
    api_key=api_key,
    type=ModelType.Chat,
    model_provider="openai",
    model="gpt-4.1",
    max_retries=20,
)

chat_model = ModelManager().get_or_create_chat_model(
    name="lazy_search",
    model_type=ModelType.Chat,
    config=chat_config,
)

print(f"Chat model initialized: {chat_config.model}")

## Prepare Search Data

Create the data container for LazySearch. The text chunks need to have `id` and `text` columns.

In [ ]:
# Prepare text chunks for LazySearch
text_chunks = text_unit_df[["id", "text"]].copy()

# Add community_id if available from the dataframe, otherwise use default
if "community_id" in text_unit_df.columns:
    text_chunks["community_id"] = text_unit_df["community_id"]
else:
    text_chunks["community_id"] = "default"

# Create LazySearchData
search_data = LazySearchData(text_chunks=text_chunks)

print(f"Prepared {len(text_chunks)} text chunks for search")

## Configure LazySearch

LazyGraphRAG provides three presets for different cost/quality tradeoffs:

| Preset | Budget | Use Case |
|--------|--------|----------|
| `z100` | 100 | Quick, low-cost queries |
| `z500` | 500 | Balanced (default) |
| `z1500` | 1500 | High-quality, thorough search |

In [ ]:
# Using Z500 preset (balanced)
config = LazySearchConfig.from_preset("z500")

print(f"Configuration:")
print(f"  Relevance budget: {config.relevance_budget}")
print(f"  Top-k chunks: {config.top_k_chunks}")
print(f"  Relevance threshold: {config.relevance_threshold}")
print(f"  Max depth: {config.max_depth}")

## Create LazySearch Instance

In [ ]:
search = LazySearch(
    model=chat_model,
    config=config,
    data=search_data,
)

print("LazySearch instance created")

## Run a Search Query

Let's run a sample query and examine the results.

In [ ]:
query = "What are the main themes and events described in the documents?"

result = await search.search(query)

print("=" * 80)
print("RESPONSE")
print("=" * 80)
print(result.response)

## Examine Search Metrics

LazySearch provides detailed metrics about the search process.

In [ ]:
print(f"Search Metrics:")
print(f"  Completion time: {result.completion_time:.2f}s")
print(f"  Iterations used: {result.iterations_used}")
print(f"  Chunks processed: {result.chunks_processed}")
print(f"  Budget used: {result.budget_used}")
print(f"  Claims extracted: {result.claims_extracted}")
print(f"  Relevant sentences: {result.relevant_sentences}")

## Using Different Presets

Let's compare results with different budget presets.

In [ ]:
# Z100 - Low budget, faster
config_z100 = LazySearchConfig.from_preset("z100")
search_z100 = LazySearch(model=chat_model, config=config_z100, data=search_data)

result_z100 = await search_z100.search("What is the main conflict in the story?")

print("Z100 Results:")
print(f"  Time: {result_z100.completion_time:.2f}s")
print(f"  Budget used: {result_z100.budget_used}")
print(f"  Response length: {len(result_z100.response)} chars")

In [ ]:
# Z1500 - High budget, more thorough
config_z1500 = LazySearchConfig.from_preset("z1500")
search_z1500 = LazySearch(model=chat_model, config=config_z1500, data=search_data)

result_z1500 = await search_z1500.search("What is the main conflict in the story?")

print("Z1500 Results:")
print(f"  Time: {result_z1500.completion_time:.2f}s")
print(f"  Budget used: {result_z1500.budget_used}")
print(f"  Response length: {len(result_z1500.response)} chars")

## Custom Configuration

You can also create custom configurations for fine-tuned control.

In [ ]:
custom_config = LazySearchConfig(
    relevance_budget=300,
    relevance_threshold=6.0,  # Higher threshold = stricter filtering
    max_depth=2,
    sufficient_relevance_count=30,
    include_citations=True,
)

search_custom = LazySearch(
    model=chat_model,
    config=custom_config,
    data=search_data,
)

result_custom = await search_custom.search("Who are the key characters?")

print("Custom Config Results:")
print(f"  Time: {result_custom.completion_time:.2f}s")
print(f"  Budget used: {result_custom.budget_used}")
print("\nResponse:")
print(result_custom.response)

## Using from_preset Factory Method

For convenience, you can use the `from_preset` factory method to create a LazySearch instance directly.

In [ ]:
# Quick setup using from_preset
quick_search = LazySearch.from_preset(
    preset="z500",
    model=chat_model,
    data=search_data,
)

result = await quick_search.search("Summarize the key events.")
print(result.response)

## Accessing Context Data

The search result includes detailed context data for analysis.

In [ ]:
# Access context data
if result.context_data:
    print("Context data keys:", list(result.context_data.keys()))
    
    # Show claims if available
    if "claims" in result.context_data:
        claims_df = result.context_data["claims"]
        print(f"\nExtracted claims: {len(claims_df)}")
        if not claims_df.empty:
            print(claims_df[["statement", "confidence"]].head())

## Best Practices

1. **Choose the right preset**:
   - `z100`: Quick FAQs, simple factual questions
   - `z500`: General questions, analysis tasks
   - `z1500`: Complex questions, detailed analysis

2. **Adjust threshold**: Higher `relevance_threshold` (6-8) for precision, lower (3-5) for recall

3. **Monitor budget**: Check `budget_used` vs `relevance_budget` to understand cost

4. **Use citations**: Set `include_citations=True` for traceable responses

## Comparison with Other Search Methods

| Feature | LazySearch | GlobalSearch | LocalSearch |
|---------|------------|--------------|-------------|
| Cost | Low (~1/100) | High | Medium |
| Speed | Fast | Slow | Medium |
| Data Required | Text chunks | Community reports | Full graph |
| Best For | General queries | Dataset summaries | Entity-specific |